In [ ]:
lang_rankings = [
    {
        "name": "document:\"object-oriented\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\" AND NOT document:\"functional\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\"^50 AND NOT document:\"functional\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\" AND NOT document:\"function\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\" AND NOT document:\"method\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\"^50 AND NOT document:\"method\"",
        "expected_langs": expect_rank
    },
    {
        "name": "paradigm:\"object-oriented\"",
        "expected_langs": expect_rank
    },
    {
        "name": "paradigm:\"object-oriented\" AND NOT paradigm:\"functional\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\"~3",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object oriented programming\"^20",
        "expected_langs": expect_rank
    },
    {
        "name": "document:(\"object-oriented\" OR \"imperative\") AND NOT document:\"functional\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"class-based\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"class-based\" AND document:\"inheritance\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\" AND document:\"encapsulation\" AND NOT document:\"functional\"",
        "expected_langs": expect_rank
    },
    {
        "name": "document:\"object-oriented\" AND document:\"polymorphism\"",
        "expected_langs": expect_rank
    }
]